<a href="https://colab.research.google.com/github/dansecret/ProjectCapstone/blob/main/ProjectCapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tempfile
import keras 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
from keras.preprocessing import image

In [3]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [10]:
from google.colab import drive
drive.mount('gdrive/My Drive/Project Capstone/')

ValueError: ignored

In [ ]:
! kaggle datasets download -d techsash/waste-classification-data

In [ ]:
! unzip waste-classification-data.zip

In [ ]:
base_dir="/content/DATASET"
train_dir=os.path.join(base_dir,"TRAIN")
test_dir=os.path.join(base_dir,"TEST")

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.1,
                                 horizontal_flip=True)

validation_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(150,150),
                                                  batch_size=64,
                                                  class_mode='binary')

validation_generator=validation_datagen.flow_from_directory(test_dir,
                                                            target_size=(150,150),
                                                            batch_size=64,
                                                            class_mode='binary')

In [ ]:
pretrain_1=InceptionV3(input_shape=(150,150,3),include_top=False,weights='imagenet')
for layer in pretrain_1.layes:
  layer.trainable = False
last_layer = pretrain_1.get_layer('mixed7')

In [ ]:
pretrain_2=MobileNet(input_shape=(150,150,3),include_top=False, weight='imagenet')
for layers_1 in pretrain_2.layes:
  layers_1.trainable=False

In [ ]:
model = tf.keras.models.Sequintal([
    tf.keras.layers.Conv2D
])